This is a *concept* for what the `photutils.psf` might look like.  Does not work (at least as of June 2016 when it was written)

In [ ]:
from astropy import units as u
from astropy.nddata import NDData
from photutils import psf, background, daofind

In [ ]:
from stsci.jwst.io import load_nircam
from stsci.jwst import jwst_phot_tools

# Load up the data 

In [ ]:
program = load_nircam('my_cycle1_data_because_all_my_friends_were_on_the_TAC.asdf')
images = program[program.filter == 'F070W'].images

`program` is something that behaves like an association table, but has a fairly easy-to-use interface for selecting individual exposures, like how it's used here.

`images` is a *list* of [NDData](http://docs.astropy.org/en/stable/api/astropy.nddata.NDData.html) objects. 

In [ ]:
stacked_image = program.drizzled_image

For the above I'm assuming there's a pipeline step that does combining on stacks in a reasonably sensible way (which I'm speculating to be drizzle-like.  Many science users will want to do that themselves, which of course they should be able to do if they want to (presumably inside this notebook).  But either way, `stacked_image` should come out as a single `NDData` object.

# Inital prep work (as needed) 

In [ ]:
full_bkgs = [background.Background(image, ...) for image in images]
images_sub = [image - full_bkg for img, full_bkg in zip(images, full_bkgs)]

In [ ]:
stacked_bkg = background.Background(stacked_image, ...)
stacked_sub = stacked_image - stacked_bkg

It might need to be a *bit* more complicated than the above to get science-quality subtraction... but ideally something nearly that simple

Alternatively, the output of `load_nircam` might yield an *already-subtracted* `NDData`... and also a `background.Background` object that the user can add back in of they want the non-subtracted version.

# Get the PSF, or derive it if needed

## The "easy" way

In [ ]:
psf_model = jwst_phot_tools.get_psf(image)

`psf_model` above is some kind of astropy.modeling.models 2D model.  It follows the interface described at https://photutils.readthedocs.io/en/latest/api/photutils.psf.psf_photometry.html#photutils.psf.psf_photometry
 
`image` already contains the metadata (equivalent to the fits header keywords), and `get_psf` does some kind of fancy black magic based on that + an optical model of JWST or whatever to figure out what the PSF is. 

## The "hard" way 

This method identifies and then fits PSF stars.  It does the finding on the stacked image, but fitting PSFs on individual exposures.  Detection could also be done on individual exposures, but that just adds a bunch of for loops here, which is a bit harder to read.

In [ ]:
from photutils import daofind
from photutils import aperture_photometry
from astropy.stats import sigma_clipped_stats

#### Find sources to be possible PSF stars - for this example we'll use daofind, but whatever could be given 

In [ ]:
mean, median, std = sigma_clipped_stats(combined_image.data, sigma=3.0, iters=5) 
sources = daofind(combined_image, fwhm=5, threshold=5.*std)

A slightly more convenient version of the above cell, not yet supported in the current photutils:

In [ ]:
sources = daofind(stacked_sub, fwhm=0.1*u.arcsec, threshold=5.*std)  
# the fwhm argument would know to use `stacked_sub.wcs` to determine the pixel scale - daofind doesn't do that yet

#### Do quick aperture photometry 

In [ ]:
positions = [(row['xcentroid'], row['ycentroid']) for row in sources]
apertures = CircularAperture(positions, r=15) 
ap_phot = aperture_photometry(stacked_sub, apertures)

This also is not-yet-supported (?), but probably should be:

In [ ]:
apertures = CircularAperture(sources, r=15)
# CircularAperture currently doesn't know how to swallow tables that come out of the star-finders, but should
ap_phot = aperture_photometry(stacked_sub, apertures)

In [ ]:
# maybe this should autmoatically go into aperture_photometry?
ap_phot['instrumental_mags'] = u.Magnitude(ap_phot['aperture_sum']*u.count)
# or maybe JWST will provide:
ap_phot['calibrated_mags'] = jwst_phot_tools.calibrate(ap_phot['aperture_sum'])

#### Now pick out the PSF stars

In [ ]:
psf_stars = psf.pick_stars(ap_phot, image_sub, nstars=10, min_seperation=50*u.pixel, psfrng=(10*u.mag, 12*u.mag))
# could give `psfrng` in units of counts, if desired

`psf_stars` is a Table that is a subset of `ap_phot`

In [ ]:
for star in psf_stars:
    # this already exists in NDData
    cutout = nddata.Cutout2D(image_sub.data, (star['xcentroid'], star['ycentroid']), size=100*u.pixel)
    plt.imshow(cutout.data)
    
# might want to provide a simple quick function to do the same as the above along the lines of:
psf.show_psf_cutouts(psf_stars, image_sub, size=100*u.pixel)

If you don't like any of them, just eliminate them from the table like this:

In [ ]:
del psf_stars[3]

#### And build the PSF

In [ ]:
psf_model = jwst_phot_tools.NIRCAMPSF.make(psf_stars, images_sub)
# this would yield whatever kind of PSF the NIRCAM folks decide is "best" - perhaps a subclass of Jay's PSF model?

Or a user might prefer their own model for the PSF:

In [ ]:
psf_model0 = picky_users_personal_tools.SomeFancy2DModel()
psf_model = picky_users_personal_tools.psf_fitter(psf_model0, psf_stars, images_sub)

## The "intermediate" way 

In [ ]:
psf_model = psf.create_psf(images_sub, find_fwhm=0.1*u.arcsec, find_threshold='5sigma', 
                           aperture_size=0.3*u.arcsec, psftype=jwst_phot_tools.JWSTPSF)

Would do all the steps as above, but just in a single easy-to-call function.

# Now do the actual PSF photometry

## Forced photometry 

This assumes you went the hard way, which has done source-finding already, using the coadded/stacked image. Effectively that's doing forced-photometry, which is conceptually straightforward, but requires that the catalogs be  

In [ ]:
# the "mode" currently only supports "sequential", but presumably something like NStar will work in the near future
psf_phot_tables = []
for image_sub in images_sub:
    psf_phot_table = psf.psf_photometry(image_sub, sources, mode='nstar', model=psf_model)
    psf_phot_tables.append(psf_phot_table)

Maybe you also want local background subtraction? (this is not yet in the function):

In [ ]:
bkg = background.MMMBackground(iters=5)  # for some reason you don't trust it for the full 20 iters
bkg_region = CircularAnnulus(r_in=15, r_out=30)  

psf_phot_tables = []
for image_sub in images_sub:
    psf_phot_table = psf.psf_photometry(image_sub, sources, mode='nstar', model=psf_model, 
                                        background_sub=bkg, background_aperture=bkg_region)
    psf_phot_tables.append(psf_phot_table)

In [ ]:
psf_phot_table = psf.combine_phot_tables(psf_phot_tables)

The function above takes the single-exposure tables and adds up all the `flux_fit`'s for matching objects.  Lots of prior art in that, but also lots of ways to go wrong.  This is possibly better for some ground-based datasets (where distortion and such can change a lot from exposure-to-exposure), but is probably not good for cases where you can do simultaneous matching.  It's probably fine (better?) for reasonably high S/N stars, though.

## Simultaneous fitting 

If the telescope is fairly stable over multiple exposures/dithers, it's probably best to instead fit multiple images *simultaneously*.

In [ ]:
psf_phot_table = []
psf_phot_table = psf.psf_photometry(images_sub, sources, model=psf_model, mode='nstar', )

this does not currently work in `psf_photometry`, but it *could*  be changed to interpret "list of NDData" as  indicating that simultaneous fitting is requested.  The resulting table would look just like the single-fit version, but of course the actual fit process would be very different.

In principal it does *not* require a separate fitter/model, however.  Instead, the fitter should be given *multiple* images and matching `x` values in one go.  This requires careful documentation of how the models are written, though: e.g., the centers need to be in "world" coordinates, even though the fitting is done in pixel coordinates.

# Next is the science 

In [ ]:
calibrated_mags = jwst_phot_tools.calibrate(psf_phot['flux_fit'])

In [ ]:
plt.hist(calibrated_mags)  # And there's your luminosity function!  

...

...

...

...

In [ ]:
win_nobel_prize()

...